# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [ ]:
#! pip install -q schedule pytest # установка библиотек, если ещё не

In [1]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import re
import time
from pathlib import Path

import schedule
import requests
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [8]:
def get_book_data(book_url: str) -> dict:
    """
    Функция возвращает информацию о книге по её URL
    вида https://books.toscrape.com...
    Для этого отправляет HTTP-запрос по указанному адресу страницы книги,
    извлекает основные данные (название, автора, описание, рейтинг и т.д.)
    и собирает их в словарь.
    Args:
        book_url (str): URL адрес по которому находится описание книги
    Returns:
        dict: Словарь с полями:
            - "Title" (str): Название книги.
            - "Price" (str): Стоимость книги.
            - "Description" (str): Описание книги.
            - "Rating" (int): Средняя оценка.
            - "Availability" (int): Наличие
            и дополнительные характеристики из таблицы Product Information
    """
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    response = requests.get(book_url, timeout=15)
    response.raise_for_status()
    return parse_book(response.text)


def parse_book(text: str) -> dict:
    """
    Функция осуществляет непосредственный разбор переданного текста HTML
    страницы, выделяет основные параметры книги и возвращает их в виде
    словаря.
    Args:
        text: html страницы
    Returns:
        dict: Словарь с полями:
            - "Title" (str): Название книги.
            - "Price" (str): Стоимость книги.
            - "Description" (str): Описание книги, если доступно.
            - "Rating" (int): Средняя оценка, если указана.
            - "Availability" (int): Наличие
            и дополнительные характеристики из таблицы Product Information
    """
    soup = BeautifulSoup(text, "html.parser")
    article = soup.find("article",
                        attrs={"class": "product_page"})
    if article is None:
        raise ValueError("Не корректная структура страницы: отсутствует тэг article.")
    book_info = {"Title": article.find("h1").text.strip()}

    price_tag = (article.find("p",
                          attrs={"class": "price_color"}))
    # Убираем Â перед ценой
    price = price_tag.get_text(strip=True).replace("Â", "")
    book_info["Price"] = price
    rating_tag = article.find("p",
                              attrs={"class": "star-rating"})
    rating_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
    if (rating_tag and rating_tag.get("class") and
            len(rating_tag["class"]) > 1):
        book_info["Rating"] = rating_map.get(rating_tag["class"][1], 0)
    avlb_tag = article.find("p",
                              attrs={"class": "instock availability"})
    if avlb_tag:
        avlb_text = avlb_tag.get_text(strip=True)
        match = re.search(r"\d+", avlb_text)
        if match:
            book_info["Availability"] = int(match.group())
    description_tag = article.find("div",
                                   attrs={"id": "product_description"})
    if description_tag:
        book_info["Description"] = (description_tag.
                                    find_next_sibling('p').text.strip())
    extra_table = article.find("table",
                               attrs={"class": "table table-striped"})
    rows = extra_table.find_all("tr")
    for row in rows:
        # Убираем дублирующиеся данные
        if not book_info.get(row.find("th").text.strip(), None):
            book_info[row.find("th").text.strip()] = (
                row.find("td").text.strip()).replace("Â", "")
    return book_info
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [9]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'Title': 'A Light in the Attic',
 'Price': '£51.77',
 'Rating': 3,
 'Availability': 22,
 'Description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [14]:
def scrape_books(is_save=True, from_page=None, to_page=None):
    """
    Собирает данные о книгах с сайта https://books.toscrape.com
    Функция проходит по страницам каталога книг (либо по всем доступным,
    либо в указанном диапазоне), извлекает ссылки на книги, получает
    данные по каждой книге и формирует список результатов. При
    необходимости данные
    могут быть сохранены во внешний файл.
    Args:
        is_save (bool, optional): Если True, результат будет сохранён
            в файл books_data.txt. По умолчанию True.
        from_page (int | None, optional): Номер первой страницы для анализа.
            Если None, обработка начинается с первой доступной страницы.
        to_page (int | None, optional): Номер последней страницы для анализа.
            Если None, обработка ведётся до последней доступной страницы.
    Returns:
        list: Список словарей, где каждый словарь содержит данные о книгах.
    """
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    about_books = []
    interval = get_interval(from_page, to_page)
    URL_PREFIX = "https://books.toscrape.com/catalogue/"
    for page_num in range(interval[0], interval[1] + 1):
        url = f"{URL_PREFIX}page-{page_num}.html"
        list_books = get_books_urls(url)
        for book_url in list_books:
            try:
                about_books.append(get_book_data(book_url))
            except Exception as e:
                print(book_url, e)
    if is_save:
        write_to_file(about_books)
    return about_books


def get_books_urls(book_catalogue_url: str) -> list:
    """
    Извлекает ссылки на отдельные книги со страницы каталога.
    Функция отправляет запрос к странице каталога книг, парсит HTML
    и собирает все ссылки, ведущие на страницы отдельных книг.
    Args:
        book_catalogue_url (str): URL страницы каталога книг.
    Returns:
        list: Список строковых URL, каждый из которых ведёт на страницу книги.
    """
    response = requests.get(book_catalogue_url, timeout=15)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    section = soup.find("section")
    rows = section.find_all("h3")
    books_url = []
    URL_PREFIX = "https://books.toscrape.com/catalogue/"
    for row in rows:
        books_url.append(URL_PREFIX + row.find("a").get("href").strip())
    return books_url


def write_to_file(about_books: list) -> None:
    """
    Запись полученной информации в файл
    Args:
        about_books (list): данные о книгах
    """
    OUTPUT_FILE_NAME = "./artifacts/books_data.txt"
    Path(OUTPUT_FILE_NAME).parent.mkdir(parents=True, exist_ok=True)
    with open(OUTPUT_FILE_NAME, "w", encoding="utf-8") as f:
        for book_info in about_books:
            for key, value in book_info.items():
                f.write(f'{key}  {value}')
                f.write('\n')


def get_page_counts() -> int:
    """
    Определяет общее количество страниц в каталоге.
    Функция обращается к первой странице каталога находит элемент,
    содержащий информацию о количестве страниц, и извлекает это число.
    Используется при пагинации для последовательного обхода всех
    страниц каталога.
    Returns:
        int: Количество страниц каталога.
    """
    URL_PREFIX = "https://books.toscrape.com/catalogue/"
    url = f"{URL_PREFIX}page-1.html"
    response = requests.get(url, timeout=15)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    pages_text = soup.find("ul",
                               attrs={"class": "pager"}).text.strip()
    return int(re.search("of (\\d+)", pages_text).group(1))


def get_interval(from_page: int, to_page: int) ->  tuple[int, int]:
    """
    Возвращает диапазон страниц для обработки.
    Функция проверяет корректность границ диапазона (номер начальной
    и конечной страниц), а если верхняя граница не указана, получает ее
    запросив информацию с сайта, если нижняя не указана ставим по
    умолчанию 1.
    Args:
        from_page (int): Номер первой страницы (включительно).
        to_page (int): Номер последней страницы (включительно).
    Returns:
        tuple[int, int]: кортеж from_page, to_page.
    Raises:
        ValueError: если from_page > to_page или если верхняя граница
        to_page, превышает максимально доступную
    """
    if from_page is None:
        from_page = 1
    max_page_number = get_page_counts()
    if to_page is None:
        to_page = max_page_number
    if from_page > to_page:
        raise ValueError("Ошибка, должно быть from_page <= to_page",
                         from_page, to_page)
    if to_page > max_page_number:
        raise ValueError(f"Ошибка, максимальная доступная "
                         f"страница {max_page_number})")
    return from_page, to_page

    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [15]:
# Проверка работоспособности функции
res = scrape_books(is_save=True, from_page=25, to_page=27) # Допишите ваши
# аргументы
print(type(res), len(res)) # и проверки

<class 'list'> 60


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [16]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
poll_interval_seconds = 50
schedule.every().day.at("19:00").do(scrape_books)
print("Планировщик запущен. Ожидаю 19:00 каждый день... (Ctrl+C для остановки)")
try:
    while True:
        schedule.run_pending()
        time.sleep(poll_interval_seconds)  # проверяем расписание не постоянно, а раз в N секунд
except KeyboardInterrupt:
    print("\n Остановлено пользователем.")
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

Планировщик запущен. Ожидаю 19:00 каждый день... (Ctrl+C для остановки)

 Остановлено пользователем.


## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [18]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest tests/test_scraper.py

============================= test session starts =============================
platform win32 -- Python 3.11.9, pytest-8.4.2, pluggy-1.6.0
rootdir: C:\MIPT\Projects\MIPTTask\module7
plugins: anyio-4.11.0
collected 8 items

tests\test_scraper.py ........                                           [100%]

============================= 8 passed in 53.39s ==============================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```